In [13]:
from pydantic_ai import Agent
import nest_asyncio
from pathlib import Path
from dotenv import load_dotenv
import chromadb
import pandas as pd
import re
from src.utilities.constants import USER_QUERY_COLLECTION_NAME, SPARQL_QUERY_COLLECTION_NAME
nest_asyncio.apply()
load_dotenv()

<frozen abc>:106: RuntimeWarning: coroutine 'websearch' was never awaited
<frozen abc>:110: RuntimeWarning: coroutine 'websearch' was never awaited
c:\Users\LorenzoFilitti\OneDrive - Avvale S.p.A\Desktop\my_git\SparqlAgent\agent_venv\Lib\site-packages\pygments\regexopt.py:26: RuntimeWarning: coroutine 'websearch' was never awaited
  def regex_opt_inner(strings, open_paren):


KeyboardInterrupt: 

In [37]:
def update_collection(create_collections : bool):
    file_path = Path.cwd() / "querySparql.xlsx"
    df = pd.read_excel(file_path, sheet_name="Foglio1", names=["question", "sparql_query"])
    ids = []
    queries = []
    for row in df.itertuples():
        user_query = re.sub("\n", " ", row[1])
        user_query = re.sub("\t", " ", user_query)

        s_query = re.sub("\n", " ", row[2])
        s_query = re.sub("\t", " ", s_query)

        queries.append({"user_query": user_query, "sparql_query": s_query})
        ids.append(f"ID_{row[0]}")

    
    client = chromadb.HttpClient(host='localhost', port=8000)
    if create_collections:
        client.create_collection(USER_QUERY_COLLECTION_NAME)
        client.create_collection(SPARQL_QUERY_COLLECTION_NAME)

    u_query_collection = client.get_collection(name=USER_QUERY_COLLECTION_NAME)
    s_query_collection = client.get_collection(name=SPARQL_QUERY_COLLECTION_NAME)


    for i, query in zip(ids, queries):
        u_query_collection.add(
            documents=query.get("user_query"),
            ids=i
        )
        s_query_collection.add(
            documents=query.get("sparql_query"),
            ids=i
        )

    return f"Collections updated\nCollections size: {u_query_collection.count(), s_query_collection.count()}"

update_collection(create_collections=False)

C:\Users\LorenzoFilitti\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████████| 79.3M/79.3M [06:43<00:00, 206kiB/s]   


'Collections updated\nCollections size: (88, 88)'

In [1]:
def decorator(func):
    def wrapper(*args, **kwargs):
        print(f"args = {args}")
    return wrapper


@decorator
def test(query):
    pass

In [27]:
import asyncio
from SPARQLWrapper import SPARQLWrapper2, JSON
import requests
import aiohttp
from bs4 import BeautifulSoup

In [76]:
query = "select ?author where {?doc a powla:Document; dcterms:creator ?author} limit 5"
params = {"query": query, "format": "json"}
endpoint = "https://lila-erc.eu/sparql/lila_knowledge_base/sparql"
res = requests.get(endpoint, params=params).json()


In [77]:
import re
result = re.findall(r"\'http.*?\'", str(res))
print(f"{len(result)}\n{result}")

4
["'http://lila-erc.eu/data/corpora/id/authors/33616ad69195643c3e0ea58463e5029a'", "'http://www.wikidata.org/entity/Q1067'", "'http://www.wikidata.org/entity/Q549041'", "'http://www.wikidata.org/entity/Q7198'"]


In [ ]:
http://www.wikidata.org/entity/Q549041sub = [re.sub("\'", "", result) for result in result]
sub[0]

'http://lila-erc.eu/data/corpora/id/authors/33616ad69195643c3e0ea58463e5029a'

In [45]:
print(res["entities"]["Q1067"]["labels"]["en"],
res["entities"]["Q1067"]["descriptions"]["en"])

{'language': 'en', 'value': 'Dante Alighieri'} {'language': 'en', 'value': 'Florentine poet, writer, and philosopher (c.\u20091265–1321)'}


In [98]:
text = requests.get("http://www.wikidata.org/entity/Q1067").json()
k = [k for k in text["entities"].keys()]